# Directory diff

In [3]:
%pip install lxml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 52.4 MB/s eta 0:00:00:00:0100:01

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## create dummy data

In [1]:
import pandas as pd

In [4]:
tables=pd.read_html('https://en.wikipedia.org/wiki/ISO_3166-1')

In [5]:
!pwd

/workspaces/utilities


In [6]:
tables[0].head()

,Short name upper case in ISO 3166,Short name lower case in ISO 3166 Full name,Listed as independent in ISO 3166,Local short name,Dispute,Link to ISO 3166-2
0,AFGHANISTAN,Afghanistanthe Islamic Republic of Afghanistan,Yes[6],Afghānistān (ps) Afghānistān (fa),Under the unrecognized government of the Islam...,ISO 3166-2:AF
1,CHINA,China the People's Republic of China,Yes[7],Zhongguo (zh),Disputed sovereignty with Taiwan,ISO 3166-2:CN
2,CYPRUS,Cyprus the Republic of Cyprus,Yes[8],Kýpros (el) Kıbrıs (tr),Disputed sovereignty with Northern Cyprus,ISO 3166-2:CY
3,FALKLAND ISLANDS (MALVINAS),Falkland Islands (the) [Malvinas][a],No[9],Falkland Islands (the) [Malvinas] (en),Disputed sovereignty and naming,ISO 3166-2:FK
4,"PALESTINE, STATE OF","Palestine, State of the State of Palestine",No[10],Dawlat Filasţīn (ar),Disputed sovereignty,ISO 3166-2:PS


In [7]:
tables[0].to_csv('dir_diff/dir1/disputed.csv',index=False) 
tables[0].to_csv('dir_diff/dir2/disputed.csv',index=False) 

In [8]:
tables[1].head()

,English short name (using title case),Alpha-2 code,Alpha-3 code,Numeric code,Link to ISO 3166-2,Independent
0,Afghanistan[b],AF,AFG,4,ISO 3166-2:AF,Yes
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,No
2,Albania,AL,ALB,8,ISO 3166-2:AL,Yes
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Yes
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,No


In [9]:
tables[1].to_csv('dir_diff/dir1/name_code.csv',index=False) 
tables[1].to_csv('dir_diff/dir2/name_code.csv',index=False) 
tables[1].to_csv('dir_diff/dir2/~name_code.csv',index=False) 

create an additional copy of the file in one directory

In [10]:
tables[2].head()

,Edition/Newsletter,Date issued,Contents
0,ISO 3166:1974,1974,First edition of ISO 3166
1,ISO 3166:1981,1981,Second edition of ISO 3166
2,ISO 3166:1988,1988-08-15,Third edition of ISO 3166
3,ISO 3166:1994,1994-02-10,Fourth edition of ISO 3166
4,ISO 3166-1:1997,1997-09-25,First edition of ISO 3166-1 (ISO 3166 expanded...


In [11]:
tables[2].to_csv('dir_diff/dir1/version_control.csv',index=False) 
tables[2].iloc[2:].to_csv('dir_diff/dir2/version_control.csv',index=False) 

create a corrupted copy of the file in one directory

## Directory listing

In [10]:
!ls -l dir_diff/*/* > diff.txt

## Directory comparison

In [17]:
list_of_headers=['permissions', 'something','owner','group','size','month','day','time','path']
dir_listing=pd.read_csv('diff.txt', sep='\s+',names=list_of_headers)

In [18]:
def split_path(df):
    return (df
    .assign(parent_dir=df
            .path
            .str.split(pat='/',expand=True)[0],
            inner_dir=df
            .path
            .str.split(pat='/',expand=True)[1],
            filename=df
            .path
            .str.split(pat='/',expand=True)[2]
           )

    )

In [19]:
dir_listing.pipe(split_path).groupby(['inner_dir']).agg({'filename':'count','size':'sum'})

,filename,size
inner_dir,,
dir1,3,16686
dir2,4,26986


difference in file count and sum of bytes. Go deeper. Identify additional file in dir2, inconsistent file size

In [20]:
dir_listing.pipe(split_path).groupby(['filename','inner_dir']).agg({'filename':'count','size':'sum'}).unstack()

filename          size         
inner_dir               dir1 dir2     dir1     dir2
filename                                           
disputed.csv             1.0  1.0   1120.0   1120.0
name_code.csv            1.0  1.0  10391.0  10391.0
version_control.csv      1.0  1.0   5175.0   5084.0
~name_code.csv           NaN  1.0      NaN  10391.0